# Apache Iceberg with Trino (using Iceberg connector)

This notebook uses Trino's Iceberg connector to create and query the same logical table as in `01_spark_iceberg_setup.ipynb`.

Requirements:
- A running Trino at `localhost:8081` with an Iceberg catalog named `iceberg`.
- The Iceberg catalog should point to the same warehouse used by Spark.
  - Recommended: HadoopCatalog with the warehouse mounted in both services.

Example Trino catalog config (`etc/catalog/iceberg.properties`):
```
connector.name=iceberg
iceberg.catalog.type=hadoop
iceberg.catalog.warehouse=file:/warehouse
```
Ensure Trino mounts your warehouse directory at the same in-container path (e.g., `/warehouse`).

Note: This repo builds a custom Trino image that bakes configs into `/etc/trino`. Build it before starting the stack.

## Quickstart: Docker (Trino + Iceberg)

REST catalog + Hadoop storage (recommended when sharing across engines):

```bash
docker compose build trino
docker compose up -d
```


In [34]:
%pip install -q sqlalchemy-trino trino pandas


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip3.12 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [37]:
import os
for var in ("HTTP_PROXY","HTTPS_PROXY","http_proxy","https_proxy"):
    os.environ.pop(var, None)
os.environ["NO_PROXY"] = "localhost,127.0.0.1"

In [38]:
from sqlalchemy import create_engine, text
import pandas as pd

ENGINE = create_engine("trino://user@localhost:8081/iceberg?http_scheme=http")

# Test connection
with ENGINE.connect() as conn:
    print(conn.execute(text("SELECT 1")).fetchall())

OperationalError: (trino.exceptions.TrinoConnectionError) failed to execute: HTTPConnectionPool(host='localhost', port=8081): Max retries exceeded with url: /v1/statement (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x116228f80>: Failed to establish a new connection: [Errno 61] Connection refused'))
[SQL: SELECT 1]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Create and Query an Iceberg Table

This section demonstrates creating a simple Iceberg table in Trino, inserting data, and querying it.


In [ ]:
from sqlalchemy import create_engine, text
import pandas as pd

ENGINE = create_engine("trino://user@localhost:8081/iceberg?http_scheme=http")

# Create schema and table
with ENGINE.begin() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS demo"))
    # Drop table if it exists (to start fresh)
    conn.execute(text("DROP TABLE IF EXISTS demo.events"))
    conn.execute(text("""
        CREATE TABLE demo.events (
            id BIGINT,
            category VARCHAR,
            ts TIMESTAMP(6)
        ) WITH (
            format = 'PARQUET',
            partitioning = ARRAY['day(ts)']
        )
    """))
    # Insert sample data
    conn.execute(text("INSERT INTO demo.events VALUES (1, 'alpha', TIMESTAMP '2024-01-01 10:00:00')"))
    conn.execute(text("INSERT INTO demo.events VALUES (2, 'beta',  TIMESTAMP '2024-01-02 12:30:00')"))
    conn.execute(text("INSERT INTO demo.events VALUES (3, 'alpha', TIMESTAMP '2024-01-02 13:45:00')"))

print("✓ Table created and data inserted successfully!")

OperationalError: (trino.exceptions.TrinoExternalError) TrinoExternalError(type=EXTERNAL, name=ICEBERG_CATALOG_ERROR, message="Failed to load view 'events'", query_id=20251014_053636_00006_8rx9m)
[SQL: DROP TABLE IF EXISTS demo.events]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Query the Data


In [ ]:
# Query the data
with ENGINE.connect() as conn:
    df = pd.read_sql(text("SELECT * FROM demo.events ORDER BY id"), conn)
    print(df)


## Query Iceberg Metadata Tables

Iceberg provides special metadata tables that give insights into table structure, files, partitions, and history.


In [ ]:
with ENGINE.connect() as conn:
    # Files metadata
    print("=== Data Files ===")
    files_df = pd.read_sql(text('SELECT file_path, record_count, file_size_in_bytes FROM demo."events$files"'), conn)
    print(files_df)
    
    print("\n=== File Summary ===")
    summary_df = pd.read_sql(text('SELECT COUNT(*) AS num_files, SUM(record_count) AS total_rows FROM demo."events$files"'), conn)
    print(summary_df)
